# Setup and Data loading

First, import the necessary libraries and load the KITTI dataset:

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

# Load KITTI dataset
# will create this class later
# from dataset_handler import Dataset_Handler
# dataset = Dataset_Handler(sequence='00')


# Sparse Image Alignment

In [ ]:
def sparse_image_alignment(prev_img, curr_img, prev_kpts):
    # Convert images to grayscale
    prev_gray = cv2.cvtColor(prev_img, cv2.COLOR_BGR2GRAY)
    curr_gray = cv2.cvtColor(curr_img, cv2.COLOR_BGR2GRAY)
    
    # Calculate optical flow
    curr_kpts, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_kpts, None)
    
    # Filter out bad points
    good_new = curr_kpts[status == 1]
    good_old = prev_kpts[status == 1]
    
    # Estimate relative pose
    E, mask = cv2.findEssentialMat(good_new, good_old, dataset.K, cv2.RANSAC, 0.999, 1.0)
    _, R, t, _ = cv2.recoverPose(E, good_new, good_old, dataset.K)
    
    return R, t, good_new


# Feature Alignment

In [ ]:
def feature_alignment(prev_img, curr_img, prev_kpts):
    # Convert images to grayscale
    prev_gray = cv2.cvtColor(prev_img, cv2.COLOR_BGR2GRAY)
    curr_gray = cv2.cvtColor(curr_img, cv2.COLOR_BGR2GRAY)
    
    # Refine feature locations
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    refined_kpts = cv2.cornerSubPix(curr_gray, prev_kpts, (5,5), (-1,-1), criteria)
    
    return refined_kpts


# Pose and Structure Refinement

In [ ]:
def pose_structure_refinement(kpts_3d, kpts_2d, K, dist_coeffs, R, t):
    # Use PnP to refine pose
    _, rvec, tvec = cv2.solvePnP(kpts_3d, kpts_2d, K, dist_coeffs, rvec=cv2.Rodrigues(R)[0], tvec=t, useExtrinsicGuess=True)
    
    R_refined = cv2.Rodrigues(rvec)[0]
    t_refined = tvec
    
    return R_refined, t_refined


# Depth Filter

In [ ]:
def depth_filter(prev_kpts, curr_kpts, K, baseline):
    # Triangulate 3D points
    points_4d = cv2.triangulatePoints(K, K, prev_kpts.T, curr_kpts.T)
    points_3d = points_4d[:3, :] / points_4d[3, :]
    
    # Filter out points with negative depth or large reprojection error
    depths = points_3d[2, :]
    valid_mask = (depths > 0) & (depths < 100)  # Assume max depth of 100m
    
    return points_3d[:, valid_mask], valid_mask
